In [5]:
import pandas as pd
import matplotlib as mpl
import os
import requests
from tqdm import tqdm
import numpy as np
import random
tqdm.pandas()

<center>*Contents:* </center><a class="anchor" id="contents"></a>

0. [Introduction](#intro)

1.1. [Library import, references and data loading](#import)
1.2. [Preprocessing](#preprocessing)

2. [Fast analysis](#fanalysis)

3. [Deep analysis, feature engineering & selection](#da)

4.1. [Prediction preparing](#pred)

4.2. [Quality metrics](#quality-metrics)

5. [Basic Models](#basic_models)

In [25]:
# get file from github

# https://medium.com/towards-entrepreneurship/importing-a-csv-file-from-github-in-a-jupyter-notebook-e2c28e7e74a5

In [ ]:
Всем добрый день!
Помогите решить задачку. Имею следующий формат учета продаж.
Необходимо посчитать тотал по каждому клиенту. 
В Power Query задачку решил, в питоне не получается.


df = pd.read_excel('/.../Новая таблица.xlsx')
pd.DataFrame.from_records(
    df.apply(lambda x: {value:x[idx+1] for idx, (col_name, value) in enumerate(zip(df.columns, x)) if 'customer' in col_name.lower()}, axis=1).values
    ).sum(axis=0)

# create random DataFrame

In [18]:
random_created_df = pd.DataFrame(np.random.randint(0,100,size=(100000, 4)),columns=['a', 'b', 'c', 'd'])
random_created_df.shape

(100000, 4)

In [19]:
random_created_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   a       100000 non-null  int32
 1   b       100000 non-null  int32
 2   c       100000 non-null  int32
 3   d       100000 non-null  int32
dtypes: int32(4)
memory usage: 1.5 MB


In [20]:
random_created_df.describe(include='all')

,a,b,c,d
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,49.501260,49.470450,49.393940,49.598440
std,28.817661,28.886251,28.903641,28.886608
min,0.000000,0.000000,0.000000,0.000000
25%,24.750000,24.000000,24.000000,24.000000
50%,50.000000,49.000000,49.000000,50.000000
75%,74.000000,74.000000,74.000000,75.000000
max,99.000000,99.000000,99.000000,99.000000


In [21]:
# insert 10% NaN values
random_created_df = random_created_df.mask(np.random.random(random_created_df.shape) < .1)

In [22]:
random_created_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       90032 non-null  float64
 1   b       90050 non-null  float64
 2   c       89980 non-null  float64
 3   d       90027 non-null  float64
dtypes: float64(4)
memory usage: 3.1 MB


In [23]:
random_created_df.describe(include='all')

,a,b,c,d
count,90032.000000,90050.000000,89980.000000,90027.000000
mean,49.479318,49.466718,49.398422,49.553967
std,28.808855,28.889509,28.904134,28.859403
min,0.000000,0.000000,0.000000,0.000000
25%,24.000000,24.000000,24.000000,24.000000
50%,50.000000,49.000000,50.000000,50.000000
75%,74.000000,74.000000,74.000000,75.000000
max,99.000000,99.000000,99.000000,99.000000


# create custom describe

In [24]:
w_df = random_created_df.copy()

show_percent_dict = {}

for _ in w_df.columns.tolist():
    percent = (w_df[w_df[_].isna()].shape[0] / w_df.shape[0]) * 100
    show_percent_dict[_] = percent

show_percent_dict = dict(sorted(show_percent_dict.items(), key=lambda x: x[1], reverse=True))
    
print(f'Всего строк в наборе данных: {w_df.shape[0]}\n')

for k,v in show_percent_dict.items():  
    v = str(int(v)) + '%'
    print(f'{v} - доля пропусков в столбце {k}')

Всего строк в наборе данных: 100000

10% - доля пропусков в столбце c
9% - доля пропусков в столбце d
9% - доля пропусков в столбце a
9% - доля пропусков в столбце b


## Склейка выгрузок в общий файл

#### [Работа с большими наборами данных](https://habr.com/ru/company/ruvds/blog/442516/) и [<font color='red'>управление памятью. </font>](https://habr.com/ru/company/mailru/blog/336156/)

In [ ]:
# показать путь к текщей директории
os.getcwd()

In [ ]:
# указываем путь к папке с файлами, которые хотим объединить
folder = r'C:\\Users\\Asus\\GKU\\2505\\all_audio'
# создаем список с названиями файлов
files_names = os.listdir(folder)

In [ ]:
#формируем список путей к файлам
files = [os.path.join(folder,f) for f in files_names] 

all_file_frames = [] #сюда будем добавлять прочитанную таблицу 

# цикл сборки таблиц в список
for f in files:
    print('Reading %s'%f)
    tab = pd.read_excel(f)
    all_file_frames.append(tab)

# склеиваем все таблицы в списке
all_frame = pd.concat(all_file_frames,axis=0) #  axis=0 если нужно добавить таблицу снизу и axis=1 если нужно слева

## Чтение и быстрое преобразование данных:

#### 1. `value_counts` и `select_dtypes` сэкономит кучу времени:

* `df.dtypes.value_counts()`


* `df.select_dtypes(include = ['float64', 'int64'])`

Полезные аргументы `value_counts`:

* `normalize = True` – проверить частоту вместо подсчёта.


* `dropna = False` – включить пропущенные значения в статистику.


* `df['c'].value_counts().reset_index()` – преобразовать таблицу статистики в объект Pandas DataFrame.


* `df['c'].value_counts().reset_index().sort_values(by='index')` – показывать статистику, отсортированную по уникальным значениям в столбце 'c' вместо количества.

#### 2. `map` - команда для простого преобразования данных.
Определяете словарь, в котором «ключами» являются старые значения, а «значениями» – новые значения:

`level_map = {1: 'high', 2: 'medium', 3: 'low'}`

`df['c_level'] = df['c'].map(level_map)`

#### <font color='red'>Как это использовать?</font>

`all_doc.applymap(type)['BIRTH_DATE'].value_counts()
bad = all_doc[all_doc['BIRTH_DATE'].map(lambda x: type(x) == str)]
good = all_doc[all_doc['BIRTH_DATE'].map(lambda x: type(x) != str)]
all_doc_2 = bad.append(good, ignore_index=True)`

#### 3.1.  Выборка и фильтрация:

конечно можно решать задачи через `loc` `iloc`

`data.loc[:,"User"]`

`data.iloc[:5,0]`

В SQL используем SELECT * FROM… WHERE ID в («A001», «C022»,…) и получаем записи с конкретными идентификаторами. Если хотите сделать то же с помощью Python библиотеки Pandas, используйте:


`df_filter = df['ID'].isin(['A001','C022',...])`

`df[df_filter]`


#### 3.2. [numpy.where](https://numpy.org/doc/stable/reference/generated/numpy.where.html) для работы со значениями:

`numpy.where(data['Revenue']>400,1,0)`

`numpy.where(data['Revenue']<data['Expense'],1,2)`

#### [3.3. Еще 3 полезных метода](https://m.habr.com/ru/company/ruvds/blog/479276/)

## Решение задач из мира Excel

[источник](https://habr.com/ru/company/ruvds/blog/500426/)

#### 0. Рукотворные ID:

`
all_audio['дата рождения2'] = all_audio['дата рождения'].dt.strftime('%d.%m.%Y')
all_audio['ФИО_строкой'] = all_audio['фамилия'] + all_audio['имя'] + all_audio['отчество'] + all_audio['дата рождения2']
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.replace('.','',regex=False)
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.replace(' ','',regex=False)
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.replace('-','',regex=False)
all_audio['ФИО_строкой'] = all_audio['ФИО_строкой'].str.upper()
`

#### и удаление дубликатов по сценарию:

`rez_true = rez2.sort_values('дата и время приема', ascending=False)
rez_true = rez_true.drop_duplicates(subset='idn', keep='first')`

#### 1. Тернарный оператор (конструкция `if`) для создания нового признака/столбца на основе значений уже имеющиегося. Реализация с помощью `list comprehension`:

`sales['MoreThan500'] = ['Yes' if x > 500 else 'No' for x in sales['Sales']]`

#### 2. `VLOOKUP (ВПР)`, решается с помощью функции [`merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html):

`sales = pd.merge(sales, states, how='left', on='City')`

1. Первый аргумент метода merge — это исходный датафрейм.


2. Второй аргумент — это датафрейм, в котором мы ищем значения.


3. Аргумент `how` (`left, right, outer, inner`, default=`inner`) указывает на то, как именно мы хотим соединить данные.


4. Аргумент `on` указывает на переменную, по которой нужно выполнить соединение (тут ещё можно использовать аргументы `left_on` и `right_on`, нужные в том случае, если интересующие нас данные в разных датафреймах названы по-разному).

#### 3. Pivot table:


`sales.pivot_table(index = 'City', values = 'Sales', aggfunc = 'sum')`

1. Здесь мы используем метод `sales.pivot_table`, сообщая pandas о том, что мы хотим создать сводную таблицу, основанную на датафрейме sales.


2. Аргумент `index` указывает на столбец, по которому мы хотим агрегировать данные.


3. Аргумент `values` указывает на то, какие значения мы собираемся агрегировать.


4. Аргумент `aggfunc` задаёт функцию, которую мы хотим использовать при обработке значений (`mean, max, min...`).

#### 4. [Styling](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)

#### 5. Заменить PowerQwery:

**было:** входящий формат данных (один айди - несколькострок, меняются 2 последних признака)
<img src="photo_2020-05-27_12-16-44.jpg" style="width: 1000px"/>


**стало:** сделать сводную по дате на признак динамики
<img src="photo_2020-05-27_12-16-52.jpg" style="width: 1000px"/>
    
    
    
`
table = pd.pivot_table(a,
                       values=['статус пациента', 'Динамика заболевания'],
                       index=['сцепка', 'id_emias','полис омс'],
                       lumns=['Дата звонка'], aggfunc=lambda x: ' '.join(x),
                       aggfunc='first') #если на пересечении только одно текстовое значение
`

#### 6. [Смещение данных `shift`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shift.html)


Есть DataFrame. Есть колонка Date, и 2 колонки со значениями Col1 и Col2.
Задача: нужно запилить колонку Col3, которая в случае, если значение col1 == 1 и col2 == 1  (тут я все сделал, это просто), либо если значение из df['col2'].iloc[x]  == 1 и df['col1'].iloc[x+1] == 1, то проставляется 1 в Col3.


**Пример: в таблице ниже за 01/05/20 в Col2 стоит 1,  а за 01/06/20 в Col1 стоит 1, то нужно в новой колонке Col3 поставить 1 на дате 01/05/20.**


<img src="3th.jpg" style="width: 400px"/>


`if df['col1']== 1 and  df['col2'].shift(1) ==1: 1`

#### 7. `crosstab`

In [ ]:
pd.crosstab(a,b)

## Ускорения расчетов `pandas`:

In [15]:
del df, new_df

**Создаем и обрабатываем таблицу средствами `pandas`:**
<a class="anchor" id="create_df"></a>

In [26]:
# создаем таблицу из 10 строк и одного столбца

df = pd.DataFrame(
    [i for i in range(10)],
    columns=["sample_column"]
)

# используем запрос к рандомайзеру слов чтобы вернуть последовательность из 3-х слов

def function_to_apply(i):
    first = requests.get(f'https://random-word-api.herokuapp.com/word').json()
    second = requests.get(f'https://random-word-api.herokuapp.com/word').json()
    third = requests.get(f'https://random-word-api.herokuapp.com/word').json()

    return " ".join([first[0], second[0], third[0]])

In [27]:
# с помощь стандартных средств `pandas` добавим столбец со словами:

# progress_apply = apply + tqdm
%timeit df["words"] = df.sample_column.progress_apply(function_to_apply)

100%|██████████| 10/10 [00:08<00:00,  1.09it/s]


9.67 s ± 170 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
df

,sample_column,words
0,0,campo emulsify boinks
1,1,indenes documentations bamboos
2,2,bicaudal integrand immingle
3,3,tabbying hecks treenail
4,4,dermabrasions tissues apoapsides
5,5,urologic outdragging cogitos
6,6,avocational ochrous walkings
7,7,deliberatively shiploads quiffs
8,8,synecologies partiality propitiations
9,9,shillelagh astarboard minutest


In [29]:
# увеличиваю датасет в 100 раз, добавляя копии в конец
new_df = pd.concat([df] * 100, ignore_index=True)

In [37]:
new_df.shape

(1000, 2)

In [40]:
# считаем среднюю длину слова
def mean_word_len(line):
    count = [len(i) for i in line.split()]
    res = sum(count) / len(count)
    return res

def compute_avg_word(df):
    return df['words'].apply(mean_word_len)

In [48]:
%timeit compute_avg_word(new_df)

1.24 ms ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


#### 1. Multiprocessing (default pack)

In [46]:
import multiprocessing
from multiprocessing import Pool

# поределяем кол-во ядер процессора
multiprocessing.cpu_count()

4

In [47]:
pool = Pool(multiprocessing.cpu_count())

In [50]:
def apply_parallel(df, func):
    # делим датафрейм на части
    df_split = np.array_split(df, n_cores)
    # считаем метрики для каждого и соединяем обратно
    df = pd.concat(pool.map(func, df_split))
    return df

In [ ]:
%timeit apply_parallel(new_df, compute_avg_word)

* Ускорение в 2-3 раза;


* Использовать распараллеливание на маленьких данных — плохая идея, т.к накладные расходы на межпроцессорное взаимодействие превышают выигрыш по времени.

#### Создадим 2-ую таблицу в 100 000 строк и 4 колонки, заполненную случайными числами от 0 до 100:

In [5]:
104818 / 239930

0.4368690868169883

In [4]:
df2 = pd.DataFrame(np.random.randint(0,100,size=(100000, 4)),columns=['a', 'b', 'c', 'd'])
df2.shape[0]

100000

In [ ]:
# функция для создания новой колонки
def multiply(x):
    return x * 5

In [5]:
# наша функция
%timeit df2['new_col'] = df2['a'].apply(multiply)

62.6 ms ± 8.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
# встроенная имплементация Pandas
%timeit df2['new_col'] = df2['a'] * 5

1.24 ms ± 24.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [8]:
# возводим значения строки в квадрат и берем их среднее 
def square_mean(row):
    row = np.power(row, 2)
    return np.mean(row)

In [10]:
# применение:
%timeit df2['new_col'] = df.apply(square_mean, axis=1)

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: overflow encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until


17.8 s ± 3.96 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 2.  [`numba`](https://habr.com/ru/post/503726/)

* Возможно добиться ускорения в тысячи раз;


* Можно использовать далеко не везде, в основном для оптимизации математических операций;


* Поддерживает не все возможности `python` и `numpy`.

In [ ]:
import numba

In [7]:
# наша функция с оптимизированная `numba`
@numba.vectorize
def multiply_numba(x):
    return x * 5

# мы отдаем весь вектор значений, чтобы numba сам провел оптимизацию цикла
%timeit df2['new_col'] = multiply_numba(df2['a'].to_numpy())

767 µs ± 350 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
# numba не умеет работать с примитивами pandas (Dataframe, Series и тд.)
# поэтому мы даем ей двумерный массив numpy
@numba.njit
def square_mean_numba(arr):
    res = np.empty(arr.shape[0])
    arr = np.power(arr, 2)
    for i in range(arr.shape[0]):
        res[i] = np.mean(arr[i])
    return res

In [13]:
# применение:
%timeit df['new_col'] = square_mean_numba(df.to_numpy())

9.4 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### 3. [`pandarallel`](https://habr.com/ru/post/498904/)

In [5]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


[Выполним данную задачу](#create_df)[<font color='red'> функциями `pandarallel`:</font>](https://github.com/nalepae/pandarallel/blob/master/docs/examples.ipynb)

In [24]:
# df["sample-word"] = df.sample_column.parallel_apply(function_to_apply)